In [ ]:
import web
import json
import sys
from time import time
import glob
import logging
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
sys.argv = ["prog", "0.0.0.0:80"]


def gen_clf():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')
    lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
    n_samples, h, w = lfw_people.images.shape
    print(n_samples)
    print(h)
    print(w)
    X = lfw_people.data
    n_features = X.shape[1]
    y = lfw_people.target
    target_names = lfw_people.target_names
    n_classes = target_names.shape[0]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    #n_components = 150
    n_components = 5
    pca = PCA(n_components=n_components, svd_solver='randomized', whiten=True).fit(X_train)
    eigenfaces = pca.components_.reshape((n_components, h, w))
    X_train_pca = pca.transform(X_train)
    #X_test_pca = pca.transform(X_test)
    param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
                  'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
    clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
    clf = clf.fit(X_train_pca, y_train)
    #y_pred = clf.predict(X_test_pca)
    return clf, pca, target_names, X_test

clf, pca, target_names, X_test = gen_clf()

urls = (
    '/picfun/search', 'search',
    '/images/(.*)', 'search_result'
)
app = web.application(urls, globals())

class search_result:        
    def GET(self, tdir):
        web.header('Content-type','image/jpeg')
        fntmpl = 'C:/hackthon/data/lfw_funneled/%s/*' % tdir
        fns = glob.glob(fntmpl)
        fn = fns[np.random.randint(0, len(fns))]
        with open(fn, "rb") as fin:
            output = fin.read()
        return output
    
class search:        
    def POST(self):
        data_in = json.loads(web.data())
        print data_in["points"]
        points = data_in["points"]
        dpi = 250
        fig = plt.figure(frameon=False, dpi=dpi)
        fig.set_size_inches(250 / dpi, 250 / dpi)
        ax = plt.Axes(fig, [0., 0., 1.0, 1.0])
        ax.set_xlim([0, 250])
        ax.set_ylim([0, 250])
        ax.set_axis_off()
        fig.add_axes(ax)
        for (x, y) in points:
            ax.scatter(x=[x], y=[249-y], c='k', s=5)
        fig.canvas.draw()
        fig.savefig('C:/hackthon/data/figure.png')
        data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
        image = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        slice_=(slice(70, 195), slice(78, 172))
        resize=0.4
        from scipy.misc import imresize
        face = np.asarray(image[slice_], dtype=np.float32)
        face /= 255.0  # scale uint8 coded colors to the [0.0, 1.0] floats
        if resize is not None:
            face = imresize(face, resize)
        face = face.mean(axis=2)
        img = face.reshape(1, -1)
        #img = X_test[[20],:]
        x_pca = pca.transform(img)
        print x_pca.sum()
        y_pred = clf.predict(x_pca)
        print y_pred[0]
        tname = target_names[y_pred[0]]
        pyDict = {'path': '/images/' + tname.replace(' ', '_'), 'name': tname}
        web.header('Content-Type', 'application/json')
        return json.dumps(pyDict)
    
class search_text:        
    def GET(self):
        output = 'my face'
        return output

if __name__ == "__main__":
    app.run()

2017-08-06 12:41:58,609 Loading LFW people faces from C:\Users\sqin2\scikit_learn_data\lfw_home


1288
50
37
http://0.0.0.0:80/
[[53, 53], [53, 50], [56, 48], [57, 46], [58, 45], [61, 43], [63, 42], [67, 41], [70, 41], [77, 39], [79, 40], [83, 42], [84, 43], [86, 43], [86, 42], [88, 43], [150, 46], [152, 45], [153, 44], [158, 42], [157, 42], [183, 36], [185, 36], [189, 36], [190, 36], [191, 36], [193, 36], [194, 36], [195, 36]]
7.49646031554
3


127.0.0.1:30619 - - [06/Aug/2017 12:45:37] "HTTP/1.1 POST /picfun/search" - 200 OK
127.0.0.1:30625 - - [06/Aug/2017 12:45:37] "HTTP/1.1 GET /images/George_W_Bush" - 200 OK


[[53, 53], [53, 50], [56, 48], [57, 46], [58, 45], [61, 43], [63, 42], [67, 41], [70, 41], [77, 39], [79, 40], [83, 42], [84, 43], [86, 43], [86, 42], [88, 43], [150, 46], [152, 45], [153, 44], [158, 42], [157, 42], [183, 36], [185, 36], [189, 36], [190, 36], [191, 36], [193, 36], [194, 36], [195, 36], [123, 65], [121, 65], [120, 68], [120, 70], [117, 73], [116, 75], [116, 81], [116, 83], [116, 84], [113, 92], [112, 98], [112, 102], [112, 103], [112, 105], [112, 111], [112, 116], [113, 123], [115, 125], [116, 126], [116, 127], [118, 128], [120, 129], [124, 132], [129, 132], [135, 130], [136, 129], [136, 128], [65, 157], [67, 158], [70, 160], [74, 162], [76, 162], [86, 168], [161, 163], [164, 162], [166, 161], [176, 156], [178, 155], [182, 153], [185, 150], [187, 149], [187, 148], [188, 147], [186, 149], [177, 150], [162, 152], [150, 154], [147, 155], [145, 156], [142, 157], [140, 157], [138, 157], [136, 157], [129, 157], [123, 155], [120, 154], [111, 154], [109, 154]]
-0.600569748656
3

127.0.0.1:30619 - - [06/Aug/2017 12:45:43] "HTTP/1.1 POST /picfun/search" - 200 OK


...

127.0.0.1:30902 - - [06/Aug/2017 12:50:45] "HTTP/1.1 POST /picfun/search" - 200 OK


[[133, 98], [88, 102], [84, 102], [81, 102], [79, 102], [77, 104], [75, 105], [74, 105], [72, 106], [70, 107], [68, 108], [69, 107], [109, 98], [112, 98], [114, 97], [114, 96], [116, 95], [114, 95], [109, 97], [103, 98], [98, 100], [91, 101], [91, 100], [92, 100], [93, 100], [92, 101], [140, 95], [143, 95], [145, 95], [149, 95], [150, 95], [156, 95], [161, 97], [171, 98], [175, 98], [178, 99], [182, 99], [194, 107], [195, 109], [196, 109], [196, 111], [197, 113], [200, 116], [199, 118], [202, 122], [204, 125], [206, 129], [206, 133], [206, 135], [205, 139], [204, 142], [202, 147], [201, 149], [161, 157], [121, 157], [119, 157], [112, 157], [110, 157], [57, 123], [60, 129], [61, 135], [61, 137], [64, 140], [64, 142], [66, 146], [67, 147], [69, 148], [71, 151], [73, 151], [76, 152], [79, 154], [86, 156], [98, 158], [100, 159], [119, 161], [154, 161], [157, 161], [161, 161], [162, 161], [164, 161], [169, 161], [173, 161], [175, 161], [183, 158], [185, 157], [187, 156], [189, 156], [191, 1

C:\ProgramData\Anaconda2\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


0.212861619392
2


127.0.0.1:30914 - - [06/Aug/2017 12:51:00] "HTTP/1.1 POST /picfun/search" - 200 OK
127.0.0.1:30917 - - [06/Aug/2017 12:51:00] "HTTP/1.1 GET /images/Donald_Rumsfeld" - 200 OK
